In [1]:
#import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [6]:
#load the files
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))
# train_df.head()
test_df.head(4)

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,target,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0,0,527975,70914,74600,99475,N,N
1,25429,25429,6000,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0,0,34628,23460,5900,23628,N,N
2,38496,38496,3600,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0,0,23100,19183,7300,15000,N,N
3,19667,19667,20000,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0,0,56481,43817,13800,35981,N,N


In [7]:
# Separate the target column (y) from the predictive features (X) in both our train and test dfs & make a copy of its data
X_train = train_df.drop("target", axis=1).copy()
X_test = test_df.drop("target", axis=1).copy()
y_train = train_df["target"].copy()
y_test = test_df["target"].copy()

In [8]:
# Convert categorical data to numeric and separate target feature for training data
X_train = pd.get_dummies(train_df.drop(columns = ['target']))
X_train.head()
# Convert categorical data to numeric and separate target feature for testing data
X_test = pd.get_dummies(test_df.drop(columns = ['target']))
X_test.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,67991,67991,40000,0.0819,814.70,140000.0,19.75,0,1,18,...,0,0,1,0,1,1,0,1,0,1
1,25429,25429,6000,0.1524,208.70,55000.0,11.52,2,0,8,...,0,0,1,0,1,1,0,1,0,1
2,38496,38496,3600,0.1695,128.27,42000.0,6.74,0,0,6,...,0,0,1,0,1,1,0,1,0,1
3,19667,19667,20000,0.1524,478.33,100000.0,12.13,0,2,7,...,0,0,1,0,1,1,0,1,0,1
4,37505,37505,3600,0.1240,120.27,50000.0,16.08,0,3,6,...,0,0,1,0,1,1,0,1,0,1


In [9]:
# First, add dummy variables that are missing in the testing set
cols_to_add = set(X_train.columns) ^ set(X_test.columns)
# Loop through each missing column and add it to X_test with all 0s
for col in cols_to_add:
        X_test[col] = 0
        
# Set the order of the columns in X_test to match the order of columns in X_train
X_test = X_test[X_train.columns]
# Encode the target column with 1s and 0s (both train and test)
X_test = X_test[X_train.columns]
target_encoder = LabelEncoder().fit(y_train)
y_train = target_encoder.transform(y_train)
y_test=target_encoder.transform(y_test)
y_test

array([1, 1, 1, ..., 0, 0, 0])

In [10]:
# Check the shapes to make sure X_train and X_test have the same number of columns (same with y_train/y_test)
# Check that X_train and y_train have the same number of rows (same with X_test/y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12180, 94)
(4702, 94)
(12180,)
(4702,)


In [11]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', random_state=24)
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))
print(lr.score(X_train, y_train))
# # Logistic regression is 50% which is not good

0.5253083794130158
0.648440065681445


C:\Users\Muppidi\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=35, max_depth=3)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
print(rf.score(X_train, y_train))

0.5857082092726499
0.7298850574712644


In [13]:
# Scale the data
scaler = StandardScaler().fit(X_train)

# Use the scaler on X_train and X_test
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# We need to fit preprocessors to training data only! If we fit preprocessors before splitting the data, we're biasing the model with information
# from the testing set.

In [14]:
# Train the Logistic Regression model on the scaled data and print the model score
lr.fit(X_train_scaled, y_train)
print(lr.score(X_test_scaled, y_test))
print(lr.score(X_train_scaled, y_train))

0.7205444491705657
0.713136288998358


C:\Users\Muppidi\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
# Train a Random Forest Classifier model on the scaled data and print the model score
rf.fit(X_train_scaled, y_train)
print(rf.score(X_test_scaled, y_test))
print(rf.score(X_train_scaled, y_train))

rf.rf.feature_importances_

0.7054444917056572
0.7224137931034482


AttributeError: 'RandomForestClassifier' object has no attribute 'rf'